1. Взять 1 любой полученный список векторов из предыдущей лабы
2. Разделить данные на обучающее и тестовое множество
3. Выбрать метрику для релевантности поста (лайки/репосты/...). Сгруппировать все данные в несколько классов (популярный/непопулярный пост).
4. Обучить минимум 3 классификатора
5. Оценить их работу с помощью метрик
6. Выбрать наилучший классификатор


## Подготовка данных для обучения классификаторов

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# достаем np.array из векторов w2v
vect = np.load('/content/drive/MyDrive/Data/vectors_w2v.npy')
vect

array([[ 0.34327167, -0.14428928, -0.12260389, ...,  0.17937243,
         0.12531067,  0.46682388],
       [ 0.04558151, -0.35105452,  0.06702286, ..., -0.17052728,
         0.22390136,  0.95037067],
       [ 0.12200719, -0.16519648, -0.51604086, ...,  0.12793829,
         0.14654893,  0.30570376],
       ...,
       [ 0.2845983 ,  0.08891457,  0.93987155, ..., -0.2466586 ,
        -0.61692864,  0.46544695],
       [ 0.28498244,  0.14119944,  0.60914177, ..., -1.4597934 ,
         0.18226525, -0.6057591 ],
       [ 0.2795401 , -0.4048951 ,  0.8534904 , ..., -1.0581628 ,
         0.1851479 ,  1.1868566 ]], dtype=float32)

In [ ]:
# делаем список из векторов, чтобы присоединить к нашему df
vect = [i for i in vect]

In [ ]:
# создаем датафрейм
df = pd.read_csv('/content/drive/MyDrive/Data/posts_ai_telegram_cleaned.csv', encoding='utf-8')
df

,id_post,date,text,views,reactions,comments,cleaned_text
0,2017,2025-03-22 07:02:11+00:00,"Немного о том, что я, редактор Individuum'а, в...",820.0,39,1,немного редактор вообще делать красноуральск с...
1,2016,2025-03-21 16:06:01+00:00,🙅 Книга-эксперимент о мытарствах современного ...,1386.0,89,3,книга-эксперимент мытарство современный художн...
2,2015,2025-03-21 15:40:09+00:00,🥴 Новое путешествие под кожу читателя от автор...,1444.0,64,6,новый путешествие кожа читатель автор бестселл...
3,2014,2025-03-21 15:39:59+00:00,"Сегодня говорим «да» предзаказам на две книги,...",1243.0,30,0,сегодня говорить предзаказ книга который говорить
4,2013,2025-03-20 09:05:47+00:00,"💰 Ишь, какие мы деловые — попали в [**шортлист...",1643.0,59,2,ишь деловой попасть шортлист премия деловой кн...
...,...,...,...,...,...,...,...
5881,34,2017-11-15 07:51:33+00:00,Так будет выглядеть наш следующий год!,1172.0,0,0,выглядеть наш следующий год
5882,33,2017-11-15 07:49:44+00:00,В 2016 году к 12 апреля художница Анастасия Пр...,1105.0,0,0,год апрель художница анастасия просочкин нарис...
5883,31,2017-11-13 08:10:28+00:00,В далекой-далекой Коста-Рике завершилась **Все...,1047.0,2,0,далёкий-далёкий коста-рика завершиться всемирн...
5884,30,2017-11-10 10:15:37+00:00,#кино в ИТМО\n13 и 14 ноября в Университете ИТ...,1139.0,2,0,кино итмый ноябрь университет итмый пройти фес...


In [ ]:
#удаляем строки с nan
df = df.dropna(subset=['cleaned_text']).copy()
# присоединяем вектора к df
df.loc[:, 'w2v'] = vect
# считаем соотношение реакций и просмотров, это и будет нашей метрикой
df.loc[:, 'metrics'] = (df['reactions'] + df['comments'])/df['views']
# добавляем метку, которая будет показывать, популярен ли этот пост
df.loc[:, 'popularity'] = df['metrics'] >= 0.015
# посмотрим, сколько постов относится к каждой из категорий
pop = df['popularity'].value_counts()
pop

,count
popularity,
False,3995
True,1843


In [ ]:
df

,id_post,date,text,views,reactions,comments,cleaned_text,w2v,metrics,popularity
0,2017,2025-03-22 07:02:11+00:00,"Немного о том, что я, редактор Individuum'а, в...",820.0,39,1,немного редактор вообще делать красноуральск с...,"[0.34327167, -0.14428928, -0.12260389, -0.2256...",0.048780,True
1,2016,2025-03-21 16:06:01+00:00,🙅 Книга-эксперимент о мытарствах современного ...,1386.0,89,3,книга-эксперимент мытарство современный художн...,"[0.045581512, -0.35105452, 0.06702286, -0.1130...",0.066378,True
2,2015,2025-03-21 15:40:09+00:00,🥴 Новое путешествие под кожу читателя от автор...,1444.0,64,6,новый путешествие кожа читатель автор бестселл...,"[0.12200719, -0.16519648, -0.51604086, -0.3109...",0.048476,True
3,2014,2025-03-21 15:39:59+00:00,"Сегодня говорим «да» предзаказам на две книги,...",1243.0,30,0,сегодня говорить предзаказ книга который говорить,"[-0.5518699, -0.8214311, -0.83089787, -0.85314...",0.024135,True
4,2013,2025-03-20 09:05:47+00:00,"💰 Ишь, какие мы деловые — попали в [**шортлист...",1643.0,59,2,ишь деловой попасть шортлист премия деловой кн...,"[0.26547128, -0.37558037, 0.1374707, -0.554015...",0.037127,True
...,...,...,...,...,...,...,...,...,...,...
5881,34,2017-11-15 07:51:33+00:00,Так будет выглядеть наш следующий год!,1172.0,0,0,выглядеть наш следующий год,"[-0.23205486, -0.5779831, 0.32519138, -1.46604...",0.000000,False
5882,33,2017-11-15 07:49:44+00:00,В 2016 году к 12 апреля художница Анастасия Пр...,1105.0,0,0,год апрель художница анастасия просочкин нарис...,"[0.1425488, -0.091513515, 0.24499108, 0.150093...",0.000000,False
5883,31,2017-11-13 08:10:28+00:00,В далекой-далекой Коста-Рике завершилась **Все...,1047.0,2,0,далёкий-далёкий коста-рика завершиться всемирн...,"[0.2845983, 0.088914566, 0.93987155, -0.405520...",0.001910,False
5884,30,2017-11-10 10:15:37+00:00,#кино в ИТМО\n13 и 14 ноября в Университете ИТ...,1139.0,2,0,кино итмый ноябрь университет итмый пройти фес...,"[0.28498244, 0.14119944, 0.60914177, -0.971553...",0.001756,False


In [ ]:
# преобразуем столбцы с векторами и метками в списки
X = df['w2v'].to_list()
Y = df['popularity'].to_list()

In [ ]:
#разделение данных на обучающую и тестовую выборку
from sklearn.model_selection import train_test_split
#разделением наш корпус данных на обучающую и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    shuffle = True,
    random_state=42,
    stratify = None)

In [ ]:
print(len(X_train))
print(len(X_test))

4670
1168


## Логистическая регрессия

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#классификатор
clf = LogisticRegression(
        penalty = 'l2', #{‘l1’, ‘l2’, ‘elasticnet’, None}, default=’l2’
        C = 0.2, #float, default=1.0
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42,
        max_iter = 1000, #int, default=100
        n_jobs = -1, #int, default=None
)

In [ ]:
#предобученный классификатор
clf_pretrained = clf.fit(X_train, y_train)

In [ ]:
clf_pretrained

LogisticRegression(C=0.2, class_weight='balanced', max_iter=1000, n_jobs=-1,
                   random_state=42)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#результат предсказания
y_predict = clf_pretrained.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predict, digits = 4))

              precision    recall  f1-score   support

       False     0.8318    0.6806    0.7486       792
        True     0.5135    0.7101    0.5960       376

    accuracy                         0.6901      1168
   macro avg     0.6726    0.6953    0.6723      1168
weighted avg     0.7293    0.6901    0.6995      1168



In [ ]:
import joblib

In [ ]:
# Сохраняем обученный классификатор
joblib.dump(clf_pretrained, '/content/drive/MyDrive/Data/logreg.joblib')

['/content/drive/MyDrive/Data/logreg.joblib']

## Метод опорных векторов

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
clf_lsvc = LinearSVC(
        penalty = 'l2', #{‘l1’, ‘l2’}, default=’l2’
        C = 0.2, #float, default=1.0
        class_weight = 'balanced', #dict or ‘balanced’, default=None
        random_state = 42,
        max_iter = 2000 #int, default=1000
)

In [ ]:
clf_lsvc = clf_lsvc.fit(X_train, y_train)
y_predict_lsvc = clf_lsvc.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predict_lsvc, digits = 4))

              precision    recall  f1-score   support

       False     0.8295    0.6818    0.7484       792
        True     0.5126    0.7048    0.5935       376

    accuracy                         0.6892      1168
   macro avg     0.6710    0.6933    0.6710      1168
weighted avg     0.7275    0.6892    0.6986      1168



In [ ]:
joblib.dump(clf_lsvc, '/content/drive/MyDrive/Data/lsvc.joblib')

['/content/drive/MyDrive/Data/lsvc.joblib']

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_KNN = KNeighborsClassifier(
        n_neighbors = 3, #int, default=5
        weights = 'distance', #{‘uniform’, ‘distance’}, callable or None, default=’uniform’
        metric = 'euclidean', #str or callable, default=’minkowski’,

)

In [ ]:
clf_KNN = clf_KNN.fit(X_train, y_train)
y_predict_KNN = clf_KNN.predict(X_test)

In [ ]:
print(classification_report(y_test, y_predict_KNN, digits = 4))

              precision    recall  f1-score   support

       False     0.7552    0.7790    0.7669       792
        True     0.5014    0.4681    0.4842       376

    accuracy                         0.6789      1168
   macro avg     0.6283    0.6236    0.6256      1168
weighted avg     0.6735    0.6789    0.6759      1168



In [ ]:
joblib.dump(clf_KNN, '/content/drive/MyDrive/Data/knn.joblib')

['/content/drive/MyDrive/Data/knn.joblib']